In [19]:
import xml.etree.ElementTree as ET
from os import listdir
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
from difflib import SequenceMatcher

# Split text and tag

Smoker and Family_hist can have no tag

In [30]:
file_path = "E:/JHU/课程/datadesign/NLP/data/testing-RiskFactors-Complete/"
names = [f for f in listdir(file_path) if f.endswith('.xml')]
test = pd.DataFrame(np.zeros((len(names), 2), dtype=object), columns=['text', 'annotation'])
print(test.shape)
n = 0
for name in names:
    tree = ET.parse(file_path + name)
    root = tree.getroot()

    ## Get the text
    nt = re.sub('\n',' ',root[0].text)
    nt = re.sub('\t',' ',nt) 
    nt = re.sub('"',"'",nt)
    ## sample 214 has a weird character
    nt = re.sub('>','&gt;',nt) 
    nt = re.sub('<','&lt;',nt)
    ## sample 262 has a weird character
    nt = re.sub('Â',' ',nt)
    nt = re.sub('â',' ',nt)
    nt = re.sub('€',' ',nt)
    nt = re.sub('™',' ',nt)
    test['text'][n] = nt
    n+=1


(514, 2)


In [34]:
n = 0
for name in names:
    tree = ET.parse(file_path + name)
    root = tree.getroot()
    ## Get the labels
    # skip FAMILY_HIST: doesn't have a text
    #skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
    #PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI') 
    # get all labels except for PHI
    tag_list = []
    for k in range(len(root[1])):
        for m in range(len(root[1][k])):
            if root[1][k][m].attrib.keys().__contains__('text') == False:
                continue
            tag_list.append((root[1][k][m].attrib['text'],root[1][k][m].tag))
    # get PHI labels
    #for k in range(PHI,len(root[1])):
    #    tag_list.append((root[1][k].attrib['text'],root[1][k].tag))
    test['annotation'][n] = tag_list
    n+=1

test.head()
    

,text,annotation
0,Record date: 2069-04-07 Mr. Villegas...,"[(aspirin, MEDICATION), (aspirin, MEDICATION),..."
1,Record date: 2075-01-07 NAME: Ville...,"[(Hypertension, HYPERTENSION), (Hypertension, ..."
2,Record date: 2080-02-18 SDU JAR Admissi...,"[(Hyperlipidemia , HYPERLIPIDEMIA), (Hyperlipi..."
3,Record date: 2080-10-01 Reason for Visit...,"[(hyperlipidemia, HYPERLIPIDEMIA), (hyperlipid..."
4,Record date: 2083-07-20 ...,"[(Zestril,, MEDICATION), (Zestril, MEDICATION)..."


# Functions to convert dataframe into input format
## use wisely, very slow 
 Please use create_labs function instead of mark_sentences


In [35]:
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string


def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'
        
    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

## replace !!mark_sentence!! to better label the text with more than one word
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)
            
    return text

def create_data(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''

    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation): 
            #text = clean(text)
            #text_ = text    
            print(test.index[test['text']== text].tolist())    
            match_list = []
            for i in annotation: 
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))
        
            d = create_labs(text, match_list)

            #for i in d.keys():
            #    f.writelines(i + ' ' + d[i] +'\n')
            #f.writelines('\n')
            for i in range(d.shape[0]):
                f.writelines(d['word'][i] + ' ' + d['label'][i] +'\n')
            f.writelines('\n')
def main():
    ## An example dataframe.
    #data = pd.DataFrame([["Horses are too tall and they pretend to care about your feelings", [("Horses", "ANIMAL")]],
    #              ["Who is Shaka Khan?", [("Shaka Khan", "PERSON")]],
    #              ["I like London and Berlin.", [("London", "LOCATION"), ("Berlin", "LOCATION")]],
    #              ["There is a banyan tree in the courtyard", [("banyan tree", "TREE")]]], columns=['text', 'annotation'])
    data = test.copy()
    #print(data.head())
    ## path to save the txt file.
    filepath = 'E:/JHU/课程/datadesign/NLP/machine_learning/testdata.txt'
    ## creating the file.
    create_data(data, filepath)
    
    
if __name__ == '__main__':
    main()

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[102]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]
[111]
[112]
[113]
[114]
[115]
[116]
[117]
[118]
[119]
[120]
[121]
[122]
[123]
[124]
[125]
[126]
[127]
[128]
[129]
[130]
[131]
[132]
[133]
[134]
[135]
[136]
[137]
[138]
[139]
[140]
[141]
[142]
[143]
[144]
[145]
[146]
[147]
[148]
[149]
[150]
[151]
[152]
[153]
[154]
[155]
[156]
[157]
[158]
[159]
[160]
[161]
[162]
[163]
[164]
[165]
[166]
[167]
[168]
[169]
[170]
[171]
[172]
[173]
[174]
[175]
[176]
[177]
[178]
[179]
[180]
[181]
[182]
[183]
[184]


In [36]:
#read txt file
def read_txt(filename):
    with open(filename, 'r') as f:
        data = f.readlines()
    return data

list = read_txt('E:/JHU/课程/datadesign/NLP/machine_learning/testdata.txt')
#print(list)
for i in range(len(list)):
    list[i] = list[i].split()
#print(list)
dataframe = pd.DataFrame(list,columns=['text','label'])
print(dataframe.shape)
dataframe.head()


(316727, 2)


,text,label
0,Record,B-SMOKER
1,date:,O
2,2069-04-07,O
3,Mr.,O
4,Villegas,O


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
df = dataframe.fillna(method='ffill')
df.text.nunique(), df.label.nunique()

(25925, 17)

In [ ]:
df.groupby('label').size().reset_index(name='counts')

,label,counts
0,B-CAD,507
1,B-DIABETES,308
2,B-FAMILY_HIST,15
3,B-HYPERLIPIDEMIA,167
4,B-HYPERTENSION,314
5,B-MEDICATION,1283
6,B-OBESE,50
7,B-SMOKER,287
8,I-CAD,2406
9,I-DIABETES,414


In [ ]:
X = df.drop('label',axis=1)
print (X)

v = DictVectorizer(sparse=True)
X = v.fit_transform(X.to_dict('records'))
print(X)
y = df.label.values

classes = np.unique(y)
classes = classes.tolist()

              text
0           Record
1            date:
2       2069-04-07
3              Mr.
4         Villegas
...            ...
158884    Resident
158885       Pager
158886           #
158887       04873
158888       04873

[158889 rows x 1 columns]
  (0, 12421)	1.0
  (1, 17008)	1.0
  (2, 3219)	1.0
  (3, 10728)	1.0
  (4, 14144)	1.0
  (5, 19915)	1.0
  (6, 23595)	1.0
  (7, 24870)	1.0
  (8, 9343)	1.0
  (9, 19072)	1.0
  (10, 21375)	1.0
  (11, 23595)	1.0
  (12, 19271)	1.0
  (13, 23827)	1.0
  (14, 11112)	1.0
  (15, 6029)	1.0
  (16, 24763)	1.0
  (17, 25622)	1.0
  (18, 14918)	1.0
  (19, 19081)	1.0
  (20, 24211)	1.0
  (21, 19284)	1.0
  (22, 11926)	1.0
  (23, 21538)	1.0
  (24, 19284)	1.0
  :	:
  (158864, 1645)	1.0
  (158865, 13604)	1.0
  (158866, 19069)	1.0
  (158867, 15657)	1.0
  (158868, 17394)	1.0
  (158869, 25732)	1.0
  (158870, 7930)	1.0
  (158871, 9703)	1.0
  (158872, 12596)	1.0
  (158873, 14105)	1.0
  (158874, 13225)	1.0
  (158875, 8535)	1.0
  (158876, 14614)	1.0
  (158877, 9765)	1.0

In [ ]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-CAD',
 'B-DIABETES',
 'B-FAMILY_HIST',
 'B-HYPERLIPIDEMIA',
 'B-HYPERTENSION',
 'B-MEDICATION',
 'B-OBESE',
 'B-SMOKER',
 'I-CAD',
 'I-DIABETES',
 'I-FAMILY_HIST',
 'I-HYPERLIPIDEMIA',
 'I-HYPERTENSION',
 'I-MEDICATION',
 'I-OBESE',
 'I-SMOKER']

# Bunch of code for testing errors
## trash

In [ ]:
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string
text = test['text'][0]
annotation = test['annotation'][0]
#pattern = pattern.strip()
match_list = []
for i in annotation:
    a, text_ = matcher(text, i[0])
    match_list.append((a[0][0], a[0][1], i[1]))


match_list

[(1516, 1523, 'MEDICATION'),
 (1516, 1523, 'MEDICATION'),
 (1516, 1523, 'MEDICATION'),
 (1516, 1523, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (1516, 1523, 'MEDICATION'),
 (1516, 1523, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (397, 405, 'MEDICATION'),
 (410, 422, 'HYPERTENSION'),
 (410, 422, 'HYPERTENSION'),
 (410, 422, 'HYPERTENSION'),
 (410, 422, 'HYPERTENSION'),
 (410, 422, 'HYPERTENSION'),
 (410, 422, 'HYPERTENSION'),
 (410, 422, 'HYPERTENSION'),
 (410, 422, 'HYPERTENSION'),
 (410, 422, 'HYPERTENSION'),
 (3, 9, 'SMOKER')]

In [ ]:
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())
        index,num_words

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

x = create_labs(test['text'][0],match_list)

In [ ]:
x

,word,label
0,Record,B-SMOKER
1,date:,O
2,2069-04-07,O
3,Mr.,O
4,Villegas,O
...,...,...
290,reviewed,O
291,by,O
292,Attending,O
293,Provider,O


## test how to find the word using index

In [ ]:
a = s.split()
a.index('smoking')
len(re.findall(r' +',s[0:start]))
a[86],len(s[start:end].split())
#temp_str = pd.DataFrame(s.split(), columns=['word'])
#temp_str['len'] = temp_str['word'].apply(lambda x: len(x))
#temp_str['cumsum'] = temp_str['len'].cumsum()
#temp_str

AttributeError: 'DataFrame' object has no attribute 'split'

## find which label went crazy and can't match text

In [33]:
match_list = []
text = test['text'][262]
#text = "Horses are too tall and they pretend to care about your feelings"
annotation = test['annotation'][262]
for i in annotation:
    print(i)
    a, text_ = matcher(text, i[0])
    print(a) 
    match_list.append((a[0][0], a[0][1], i[1]))
    


('High cholesterol', 'HYPERLIPIDEMIA')
[(974, 990)]
('High cholesterol ', 'HYPERLIPIDEMIA')
[(974, 990)]
('High cholesterol', 'HYPERLIPIDEMIA')
[(974, 990)]
('high chol,', 'HYPERLIPIDEMIA')
[(2605, 2615)]
('High cholesterol', 'HYPERLIPIDEMIA')
[(974, 990)]
('High cholesterol ', 'HYPERLIPIDEMIA')
[(974, 990)]
('High cholesterol', 'HYPERLIPIDEMIA')
[(974, 990)]
('high chol,', 'HYPERLIPIDEMIA')
[(2605, 2615)]
('High cholesterol', 'HYPERLIPIDEMIA')
[(974, 990)]
('High cholesterol ', 'HYPERLIPIDEMIA')
[(974, 990)]
('High cholesterol', 'HYPERLIPIDEMIA')
[(974, 990)]
('high chol,', 'HYPERLIPIDEMIA')
[(2605, 2615)]
('ASA', 'MEDICATION')
[(756, 759)]
('ASA', 'MEDICATION')
[(756, 759)]
('ASA', 'MEDICATION')
[(756, 759)]
('ASA', 'MEDICATION')
[(756, 759)]
('Plavix', 'MEDICATION')
[(1094, 1100)]
('Plavix', 'MEDICATION')
[(1094, 1100)]
('ASA', 'MEDICATION')
[(756, 759)]
('ASA', 'MEDICATION')
[(756, 759)]
('ASA', 'MEDICATION')
[(756, 759)]
('ASA', 'MEDICATION')
[(756, 759)]
('Norvasc', 'MEDICATION')

IndexError: list index out of range

In [12]:
test['text'][262]
#tree = ET.parse(file_path + names[62])
#root = tree.getroot()
#root[0].text

"   Record date: 2094-08-11  CARDIOLOGY  COQUILLE VALLEY HOSPITAL    Reason for visit:     Evaluation for cardiac catheterization with Dr. Irvin Vitale.     Interval History:     Multiple risk factors for CAD including DM, HTN, CRI. Now with complaints of exertional dyspnea and fatigue. Progressed and she was evaluated by Bonnie Eaves.     8/25 Bruce protocol was postive at 9 minutes, 64% PMHR. Complaints of dyspnea. EKG with borderline changes. Nuclear images with moderate sized defect of anteroapical zone with partial reperfusion. Some scar. Mild LV dysfunction with EF 45-50% and apical dyskinesis.     Arranged for elective cardiac catheterization with Dr Vitale.     Past medical history:     CVA of left PCA territory 2093. No residual. Rx with ASA, folate, niaspan  IDDM diagnosed 30 yrs ago  CRI with creat baseline 3.7 (followed by Orlando Ernst) and recent eval by Dr Ratliff for transplant. +proteinuria with nephrotic syndrome. Donor kidney lined up.  HTN  High cholesterol  Diabeti

In [13]:
names[262]

'231-01.xml'

In [14]:
tree = ET.parse(file_path + names[0])
root = tree.getroot()
## Get the labels
# skip FAMILY_HIST: doesn't have a text
#skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
#PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
# get all labels except for PHI
tag_list = []
root[1][5][0].attrib['text']

'Atenolol'